# Project: Perception on Public Security in the United States of Mexico

## Introduction

After a recent trip to Mexico, I decided to explore the website of the National Institute of Statistics and Geography of Mexico ([INEGI](https://en.www.inegi.org.mx/default.html)) and use this project as an opportunity to learn something more about the country. 

After exploring the data available on the website, I decided to investigate the issue of security, particalarly perceived security, within the country. I chose this topic for 2 main reasons: 

1) While planning my trip, I was surprised of the concern expressed by my Mexican friends (both male and female). Also, during my trip, I noticed that my own perception of security (or lack thereof) was different, based on the city/town I was visiting. 

2) The data available on the INEGI website looks complex, and interesting enough to investigate: the survey has been iterated for a number of years (yearly, from 2010 to 2025), and it's divided by state, which allows both a temporal and a geographic analysis. 

### Project scope

The goal of this project is to investigate the following points: 

- How does perception of public security in Mexico change, based on the state? 

- Has it changed, overall, in the past 15 years? 

- What are the main concerns that influence perceived security (or lack of it)? 

## 1. Sourcing the data 

All the packages required for the project are imported.

In [1]:
import pandas as pd
import requests 
import openpyxl

### API requests and download

After exploring the webiste of INEGI, I found 2 ways of sourcing public data: 

- using the API 
- dowloading the data i .xlsx format. 

I explored both ways. 

**API requests**

The API requires a token (or key), which can be requested for free by registering with an email address. The API requests can be created using a [Query Builder](https://en.www.inegi.org.mx/servicios/api_indicadores.html) that inputs parameters such as: 

- indicator,
- geographic area,
- language (Spanish or English).

The [documentation](https://en.www.inegi.org.mx/servicios/api_indicadores.html) is quite clear and accessible, and available in Spanish and English.

However, the API has some limitations:

1. it returns responses in .json, .jsonp, or .xml, which make it hard to read and manipulate the data with Pandas (see below);
2. it only allows to request data about one area at the time, which does not allow to compare data points across regions;
3. it allows to input max 10 indicators per request, which returns a partial dataset and limits analysis.

In [2]:
# code provided by chatGPT: https://chatgpt.com/share/6950450b-d294-800f-8cb4-c585e1a7fe7f

# 1. Fetch the JSON from the API
url = "https://en.www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/6200028417,6200028418,6200028419,6200028420,6200028431,6200028430,6200028427,6200028425,6200028423,6200028433/en/00,01/true/BISE/2.0/0f18317c-23e8-90bd-c373-f2e07a38e907?type=json"
response = requests.get(url)
data = response.json()  # Convert response to Python dict

# 2. Create a DataFrame from the "Series" list
series_df = pd.json_normalize(data["Series"])

# 3. Also flatten the "OBSERVATIONS" nested list
obs_df = pd.json_normalize(
    data["Series"],
    record_path=["OBSERVATIONS"],
    meta=["INDICADOR"],
    errors="ignore"
)

# 4. Merge Series-level with Observations
df = series_df.drop(columns=["OBSERVATIONS"]).merge(
    obs_df,
    on="INDICADOR",
    how="inner"
)

# preview data 
df.head(5)

,INDICADOR,FREQ,TOPIC,UNIT,UNIT_MULT,NOTE,SOURCE,LASTUPDATE,STATUS,TIME_PERIOD,OBS_VALUE,OBS_EXCEPTION,OBS_STATUS,OBS_SOURCE,OBS_NOTE,COBER_GEO
0,6200028417,3,163,3,,,"1024,1027,1054,1171,1795,2196,2782,2822,2928,2...",18/09/2025 12:00:00 a. m.,None,2025,34.35740488780000000000,None,3,,,0
1,6200028418,3,163,3,,5255,"1024,1027,1054,1171,1795,2196,2782,2822,2928,2...",18/09/2025 12:00:00 a. m.,None,2025,18.44711426750000000000,None,3,,,0
2,6200028419,3,163,3,,5256,"1024,1027,1054,1171,1795,2196,2782,2822,2928,2...",18/09/2025 12:00:00 a. m.,None,2025,63.80192530400000000000,None,3,,,0
3,6200028420,3,163,3,,5257,"1024,1027,1054,1171,1795,2196,2782,2822,2928,2...",18/09/2025 12:00:00 a. m.,None,2025,23.72884692830000000000,None,3,,,0
4,6200028431,3,163,3,,5267,"1024,1054,1171,1795,2196,2782,2822,2928,2974,2...",18/09/2025 12:00:00 a. m.,None,2025,16.67994456540000000000,None,3,,,0


The code shows that reading the response with Pandas is not straightforward (it's not enough to use [pandas.read_json()](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html)). Morever, once the data is imported to a dataframe, it is still quite hard to read, because: 

- indicators and geographic areas are identified with IDs instead of full names, so they should be remapped.
- each API response shows limited data (only 10 values, only about 1 area). 

It would be possible to make the data complete, creating multiple requests (iterating through areas and indicators) and remapping names. However, this would add complexity to the task. For this reason, I decided the most effective way of sourcing the data was downloading the full dataset in .xlsx format. 

**Data download**

The website of INEGI allows users to explore public data and to download datasets in the format .xlsx. Files include multiple tabs: one for the data, and one or more for metadata. See: https://en.www.inegi.org.mx/app/descarga/?t=163. 

The dataset on *Perception of Public Security* is downloaded and added to the repository in the folder *data*: [Perception dataset](project/data/percepcion_english_00.xlsx). It includes 3 tabs: 

- the dataset 
- metadata for each indicator and for each year 
- metadata on each indicator. 

Pandas.read_excel is used to read the full dataset (1st tab of the file). The dataset includes all the indicators and geographic areas, including IDs and full names. 

In [3]:
# set file path 
filepath = "./data/public_security.xlsx"

# read data. See: # see: https://pandas.pydata.org/docs/reference/api/pandas.read_json.html
df = pd.read_excel(filepath, sheet_name='valor', skiprows=4)

# preview the data
df.head()

,cve_entidad,desc_entidad,cve_municipio,desc_municipio,id_indicador,indicador,2010,2011,2012,2013,...,2023/t4,2024,2024/t1,2024/t2,2024/t3,2024/t4,2025,2025/t1,2025/t2,unidad_medida
0,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,6200002198,Rate of people aged 18 and older who considere...,NaN,69536.343084,66622.674217,72322.456768,...,NaN,73621.540745,NaN,NaN,NaN,NaN,75556.050590,NaN,NaN,"Cases per 100,000 inhabitants"
1,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,6200002235,Rate of economic units that consider their nei...,NaN,NaN,5460.456292,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Cases per 10,000 economic units"
2,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,6200002237,Rate of economic units that consider their sta...,NaN,NaN,7602.826537,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Cases per 10,000 economic units"
3,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,6200002238,Rate of economic units that consider their mun...,NaN,NaN,7052.320400,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Cases per 10,000 economic units"
4,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,6200005815,Rate of people aged 18 years and over that con...,NaN,39481.723643,38674.118886,44011.361167,...,NaN,40210.785898,NaN,NaN,NaN,NaN,40462.996387,NaN,NaN,"Cases per 100,000 inhabitants"


## 2. Cleaning up the data 

## 3. Analysis  

## 4. Conclusion

## End